In [2]:
# Required for all calculations
import numpy as np
import mhs

# For timing our results
from time import time
timings = {}

# Required for scattered node layout
from scipy import interpolate, ndimage
from rbf.pde import fd

# Required to pull SHARP from jsoc
import os
from astropy.io import fits
import drms

# Required to visualize results
import pyvista as pv
%matplotlib ipympl
import matplotlib.pyplot as plt
import streamtracer

In [27]:
# First pull a SHARP image

series = 'hmi.sharp_cea_720s'
sharpnum = 7821
year=2021
month=11
day = 9
hour = 3
minute = 24
second = 0
timings['sharpStart'] = time()

tstr = '{ye}.{mo:02d}.{da:02d}_{ho:02d}:{mi:02d}:{se:02d}'.format(
        ye=year,mo=month,da=day,ho=hour,mi=minute,se=second
        )

c = drms.Client(email = 'nathanielhm@gmail.com', verbose=False)
tempdir = '.\\sharpdata'
if not os.path.exists(tempdir):
    os.mkdir(tempdir)

exp_query = '{series}[{sharpnum}][{year}.{month:02d}.{day:02d}_{hour:02d}:{minute:02d}:{second:02d}]'
fname = f'{series}.{sharpnum}.{year}{month:02d}{day:02d}_{hour:02d}{minute:02d}{second:02d}_TAI.' + '{segment}.fits'

metadata = c.query(exp_query.format(
                      series=series,sharpnum=sharpnum,
                      year=year,month=month,day=day,hour=hour,minute=minute,second=second
                  ), key=['RSUN_REF','RSUN_OBS','CDELT1','CDELT2'])
if not (os.path.exists(tempdir + '\\' + fname.format(segment='Bt')) and \
        os.path.exists(tempdir + '\\' + fname.format(segment='Br')) and \
        os.path.exists(tempdir + '\\' + fname.format(segment='Bp'))):
    request = c.export(exp_query.format(
                          series=series,sharpnum=sharpnum,
                          year=year,month=month,day=day,hour=hour,minute=minute,second=second
                      ) + '{Bt,Br,Bp}')
    request.download(tempdir)

sharpfitsT = fits.open(tempdir + '\\' + fname.format(segment='Bt'),memmap = False)
sharpfitsR = fits.open(tempdir + '\\' + fname.format(segment='Br'),memmap = False)
sharpfitsP = fits.open(tempdir + '\\' + fname.format(segment='Bp'),memmap = False)

Bt = sharpfitsT[1].data
Br = sharpfitsR[1].data
Bp = sharpfitsP[1].data

lenX = metadata.iloc[0].RSUN_REF / metadata.iloc[0].RSUN_OBS * metadata.iloc[0].CDELT1 * sharpfitsT[1].header['NAXIS1'] / 1e6
lenY = metadata.iloc[0].RSUN_REF / metadata.iloc[0].RSUN_OBS * metadata.iloc[0].CDELT2 * sharpfitsT[1].header['NAXIS2'] / 1e6
lenZ = np.max((lenX,lenY))*2

timings['sharpEnd'] = time()
print(f"SHARP pull and read completed in  {timings['sharpEnd']-timings['sharpStart']:.02f} seconds")

SHARP pull and read completed in  1.88 seconds


In [49]:
# Create a new, adaptively scattered domain in cartesian geometry
timings['nodeStart'] = time()

points = (np.linspace(lenY,0,sharpfitsR[1].header['NAXIS2']),np.linspace(0,lenX,sharpfitsR[1].header['NAXIS1']))
BxInterp = interpolate.RegularGridInterpolator(points,Bt)
BzInterp = interpolate.RegularGridInterpolator(points,Br)
ByInterp = interpolate.RegularGridInterpolator(points,-Bp)

supersmooth = ndimage.maximum_filter(abs(ndimage.gaussian_laplace(Br,1)),20)
smoothfn = interpolate.RegularGridInterpolator(points,(1-supersmooth/np.max(supersmooth.ravel())))
print(lenX,lenY)
def rfn(xyz):
    if len(xyz.shape) == 1:
        xyz = np.expand_dims(xyz,0)
    omega = 1 / lenZ;
    print(min(xyz[:,0]),min(xyz[:,1]),max(xyz[:,0]),max(xyz[:,1]))
    return smoothfn((xyz[:,0],xyz[:,1])) ** 2 * lenZ/2 * 2e-2 + lenZ/2*8e-3 * np.exp(omega*xyz[:,2]);

nodes = mhs.node_drop_3d([0,lenX,0,lenY,0,lenZ], (sharpfitsR[1].header['NAXIS2'],sharpfitsR[1].header['NAXIS1']), 1e5, rfn)
n = nodes.shape[0]
index = mhs.Index(np.where(nodes[:,0]<1e-4*lenX)[0],np.where((np.max(nodes[:,0])-nodes[:,0])<1e-4*lenX)[0],
                  np.where(nodes[:,1]<1e-4*lenY)[0],np.where((np.max(nodes[:,1])-nodes[:,1])<1e-4*lenY)[0],
                  np.where(nodes[:,2]<1e-4*lenZ)[0],np.where((np.max(nodes[:,2])-nodes[:,2])<1e-2*lenZ)[0],[]
                 )
timings['nodeBuilt'] = time()
print(f"Built nodes (n={nodes.shape[0]:.02f},zmax={max(nodes[:,2])}) in {timings['nodeBuilt']-timings['nodeStart']:.02f} seconds")
print('Determined boundary nodes with counts: ' + str(index))

Bx0 = np.expand_dims(BxInterp((nodes[index.z0,0],nodes[index.z0,1])),1)
By0 = np.expand_dims(ByInterp((nodes[index.z0,0],nodes[index.z0,1])),1)
Bz0 = np.expand_dims(BzInterp((nodes[index.z0,0],nodes[index.z0,1])),1)

# And now we can create the differentiation matrices for this domain
stencil_n = 200
order = 4
Dx  = fd.weight_matrix(nodes,nodes,stencil_n,[1,0,0],phi='phs5',order=order).tocsr()
timings['nodeDx'] = time()
print(f"Computed Dx in {timings['nodeDx']-timings['nodeBuilt']:.02f} seconds.")
Dy  = fd.weight_matrix(nodes,nodes,stencil_n,[0,1,0],phi='phs5',order=order).tocsr()
timings['nodeDy'] = time()
print(f"Computed Dy in {timings['nodeDy']-timings['nodeDx']:.02f} seconds.")
Dz  = fd.weight_matrix(nodes,nodes,stencil_n,[0,0,1],phi='phs5',order=order).tocsr()
timings['nodeDz'] = time()
print(f"Computed Dz in {timings['nodeDz']-timings['nodeDy']:.02f} seconds.")
Dxx = fd.weight_matrix(nodes,nodes,stencil_n,[2,0,0],phi='phs5',order=order).tocsr()
timings['nodeDxx'] = time()
print(f"Computed Dxx in {timings['nodeDxx']-timings['nodeDz']:.02f} seconds.")
Dyy = fd.weight_matrix(nodes,nodes,stencil_n,[0,2,0],phi='phs5',order=order).tocsr()
timings['nodeDyy'] = time()
print(f"Computed Dyy in {timings['nodeDyy']-timings['nodeDxx']:.02f} seconds.")
Dzz = fd.weight_matrix(nodes,nodes,stencil_n,[0,0,2],phi='phs5',order=order).tocsr()
timings['nodeDzz'] = time()
print(f"Computed Dzz in {timings['nodeDzz']-timings['nodeDyy']:.02f} seconds.")

18.322449731433128 6.854751781877334
0.0 0.0 18.322449731433128 6.854751781877334


ValueError: One of the requested xi is out of bounds in dimension 0

In [38]:
# Now we are ready to perform the MHS extrapolation
timings['mhsInit'] = time()
config = {'lsqr_tol':1e-5,
          'lsqr_iter':1e4,
          'maxiters':2,
          'gpu':False,
          'verbose':True
         }
# First compute the potential field
Bpot = mhs.potfield(nodes, Bz0, Dx, Dy, Dz, Dxx, Dyy, Dzz, index,
                    config=config)
timings['Bpot'] = time()
print(f"Computed potential field in {timings['Bpot']-timings['mhsInit']} seconds.")

Potential solution done: 1. Iteration 2002 returned residual 473.8674867941269
Computed potential field in 1132.3771781921387 seconds.


In [ ]:
B, r = mhs.num_mhs(np.zeros((n,1)),np.zeros((n,1)), Bx0,By0,Bz0, nodes, Dx, Dy, Dz, Dxx, Dyy, Dzz, index,
                   config=config, Binit=Bpot)
timings['Bmhs'] = time()
print(f"Computed mhs field in {timings['Bmhs']-timings['Bpot']} seconds.")

In [ ]:
fig = plt.figure()
ax = plt.subplot(2,1,1)
ax = ax.scatter(nodes[index.z0,0],nodes[index.z0,1],c=Bpot[index.z0,2],s=10)
plt.colorbar(ax)
ax = plt.subplot(2,1,2)
ax = ax.scatter(nodes[index.z0,0],nodes[index.z0,1],c=Bz0,s=10)
plt.colorbar(ax)
plt.show()

In [39]:
import importlib
importlib.reload(mhs)

In [40]:
st = streamtracer.StreamTracer(int(1e3), lenX/1e2)
timings['streamblock'] = time()
# Binterp = interpolate.RBFInterpolator(nodes,B,neighbors=60,kernel='cubic',degree=2)
Bxinterp = interpolate.NearestNDInterpolator(nodes,B[:,0])
Byinterp = interpolate.NearestNDInterpolator(nodes,B[:,1])
Bzinterp = interpolate.NearestNDInterpolator(nodes,B[:,2])
timings['interp'] = time()
print(f"Built interpolant in {timings['interp']-timings['streamblock']} seconds.")

xgrid,ygrid,zgrid = np.meshgrid(np.linspace(0,lenX,100),np.linspace(0,lenY,100),np.linspace(0,lenZ,100))
# Bgrid = Binterp(np.hstack((np.expand_dims(xgrid.ravel(),1),np.expand_dims(ygrid.ravel(),1),np.expand_dims(zgrid.ravel(),1))))
Bxgrid = Bxinterp(xgrid.ravel(),ygrid.ravel(),zgrid.ravel())
Bygrid = Byinterp(xgrid.ravel(),ygrid.ravel(),zgrid.ravel())
Bzgrid = Bzinterp(xgrid.ravel(),ygrid.ravel(),zgrid.ravel())
Bgrid = np.vstack((Bxgrid,Bygrid,Bzgrid)).transpose()
print(Bgrid.shape)
Bgrid = Bgrid.reshape((100,100,100,3))
timings['grid'] = time()

print(f"interpolated field in {timings['grid']-timings['interp']} seconds.")
st = streamtracer.StreamTracer(int(1e5),min((lenX,lenY,lenZ))/1e2)
vg = streamtracer.VectorGrid(Bgrid, grid_coords=[np.linspace(0,lenX,100),np.linspace(0,lenY,100),np.linspace(0,lenZ,100)])
xseed,yseed = np.meshgrid(np.linspace(0,lenX,20),np.linspace(0,lenY,20))
seed = np.hstack((np.expand_dims(xseed.ravel(),1),np.expand_dims(yseed.ravel(),1),np.zeros((20**2,1))))
st.trace(seed,vg)
timings['trace'] = time()
print(f"Traced {20**2} field lines in {timings['trace'] - timings['grid']} seconds.")

Built interpolant in 0.07999777793884277 seconds.
(1000000, 3)
interpolated field in 3.2917544841766357 seconds.
Traced 400 field lines in 43.85793161392212 seconds.


In [ ]:
pl = pv.Plotter()
for stream in st.xs:
    pl.add_lines(stream,color='black',width=1,connected = True)
pl.show(jupyter_backend='trame')

In [41]:
Zmesh = pv.ImageData()
Zmesh.dimensions = np.array(xgrid.shape)+1
Zmesh.origin = (0,0,0)
Zmesh.spacing = (xgrid[1,1,1],ygrid[1,1,1],zgrid[1,1,1])
Zmesh.cell_data["values"] = Bgrid[:,:,:,2].ravel('F')
sl = Zmesh.slice(normal=[0,0,1],origin=[0.1,0.1,0.1])
sl.plot()

Widget(value="<iframe src='http://localhost:55366/index.html?ui=P_0x23d2d6522c0_9&reconnect=auto' style='width…

In [34]:
sharpfitsR[1].header['NAXIS2']

318

In [35]:
sharpfitsR[1].data.shape

(318, 850)

In [36]:
np.linspace(1,0,10)

array([1.        , 0.88888889, 0.77777778, 0.66666667, 0.55555556,
       0.44444444, 0.33333333, 0.22222222, 0.11111111, 0.        ])